<img src='../img/TUD_logo.png' align='right' width='15%'></img>

[Zurück zur Übersicht MWW01 - Grundwasserbewirtschaftung mit Computermodellen](../MWW01_00_index.ipynb)   
[Zurück zur Übersicht T03 - Numerische Lösung Strömung](./MWW01_T03_00_index.ipynb)

# Numerische Lösung für Grundwasserströmung 1D für ungespannten und heterogenen Grundwasserleiter

Developed by: Thomas.Reimann@tu-dresden.de / Sophie.Pfoertner@mailbox.tu-dresden.de / Anne.Pfoertner@mailbox.tu-dresden.de / Alexander.Oy@mailbox.tu-dresden.de
<br>Last change: 2020 11 09
<br>Current state: funktional
<br>ToDo:
- Quellcode kommentieren
- Tabelle output optimieren
- Eingangswerte etvl mit ipywidgets ins Modell

Die Anwendung ist im Moment noch im Entstehen. Es werden momentan zwei Zonen für hydraulische Leitfähigkeit und Grundwasserneubildung berücksichtigt. Die Parameter (K1, K2, GWN1, GWN2) können im Moment nur direkt im Quellcode geändert werden.

In [11]:
import matplotlib.pyplot as plt
import numpy as np
from numpy import nan as NaN
from ipywidgets import *
from IPython.display import display
import pandas as pd
from IPython.display import clear_output

In [16]:
button = Button(description="Berechnen")
output = Output()
out_plot = Output()

dx = 500                # Zellweite

m = 12                  # Mächtigkeit
imax = 100              # maximale Iterationszahl
epsilon = 0.00001       # Abbruchkriterium
ncol = 11             # Anzahl der Zellen
  
#Zonierung
Anteil_Zone1 = 0.4      # Anteil der Zone 1 
K1 = 2e-4               # hydraulische Leitfähigkeit der Zone 1
K2 = 2e-4               # hydraulische Leitfähigkeit der Zone 2
GWN1 = 0                # Grundwasserneubildung der Zone 1 in mm/a
GWN2 = 0                # Grundwasserneubildung der Zone 2 in mm/a

RCH1 = GWN1/1000/365.25/24/3600        #Umrechnung der Grundwasserneubildung in m/s
RCH2 = GWN2/1000/365.25/24/3600 

K_Zone = []
RCH_Zone = []
for z in range (0, int(ncol*Anteil_Zone1)):
    K_Zone.append(K1)
    RCH_Zone.append(RCH1)
for z in range (int(ncol*Anteil_Zone1),ncol):
    K_Zone.append(K2)
    RCH_Zone.append(RCH2)

In [17]:
# Generate Dataframe and initiate the first row
col = []
leerZeile = []
for n in range (ncol):
    col.append('h'+ str(n+1))  # COLUMN HEADER, START WITH 1
    leerZeile.append(NaN)      # EMPTY ROW

global i
i = 0
global HEAD
HEAD = pd.DataFrame([leerZeile], columns=col)
global HEAD_out
HEAD_out = pd.DataFrame([leerZeile], columns=col)

In [18]:
#Randbedingungen/ Startwerte der Zellen
HEAD[col[ 0]].values[i] = 18              # Potentialhöhe Links in m
HEAD[col[-1]].values[i] = 16              # Potentialhöhe Rechts in m
HEAD_out[col[ 0]].values[i] = 18              # Potentialhöhe Links in m
HEAD_out[col[-1]].values[i] = 16              # Potentialhöhe Rechts in m

# SET INITIAL VALUE TO THE AVERAGE OF BOTH BOUNDARIES
for n in range(1,ncol-1):
    HEAD[col[n]].values[i] = (HEAD[col[0]].values[i] + HEAD[col[-1]].values[i])/2
    HEAD_out[col[n]].values[i] = "%7.3f"% (HEAD[col[n]].values[i])

In [19]:
  
def iterator(HEAD=HEAD,HEAD_out=HEAD_out, RCH_Zone=RCH_Zone, K_Zone=K_Zone, dx=dx, i=i, col=col, m=m):
    T = []
    # FIRST ITERATION
    if i==0:
        i = i + 1
        NEW_LINE = pd.DataFrame([leerZeile], columns=col) 
        HEAD = HEAD.append(NEW_LINE, ignore_index=True)
        HEAD_out = HEAD_out.append(NEW_LINE, ignore_index=True)
        HEAD[col[ 0]].values[i] = HEAD[col[ 0]].values[i-1]
        HEAD[col[-1]].values[i] = HEAD[col[-1]].values[i-1]
        HEAD_out[col[ 0]].values[i] = 18              # Potentialhöhe Links in m
        HEAD_out[col[-1]].values[i] = 16              # Potentialhöhe Rechts in m
        for n in range(0,ncol):
            T.append(K_Zone[n] * min([m, HEAD[col[n]].values[i-1]]))                     #Transmissivität
        for n in range(1,ncol-1):
            HEAD[col[n]].values[i]=(2*T[n]*T[n-1]/(T[n]+T[n-1])*HEAD[col[n-1]].values[i-1]+2*T[n]*T[n+1]/(T[n]+T[n+1])*HEAD[col[n+1]].values[i-1]+RCH_Zone[n]*dx**2)/(2*T[n]*T[n-1]/(T[n]+T[n-1])+2*T[n]*T[n+1]/(T[n]+T[n+1]))
            HEAD_out[col[n]].values[i] = "%5.2f"% (HEAD[col[n]].values[i])
        with output:
            clear_output()
            print(HEAD_out[1:])
        with out_plot:
            fig = plt.figure(figsize=(4,4))   
            ax = fig.add_subplot(1, 1, 1)
            clear_output()
            ax.plot(HEAD[-1:].T)
            plt.show()
        return HEAD, HEAD_out, i    
    else:
        i = i + 1
        max_head_change = max(abs(HEAD.iloc[-2]-HEAD.iloc[-1]))
        if max_head_change<epsilon or i==imax:
            with output:
                print('Die Iteration wurde beendet.')
        else:
            NEW_LINE = pd.DataFrame([leerZeile], columns=col) 
            HEAD = HEAD.append(NEW_LINE, ignore_index=True)
            HEAD_out = HEAD_out.append(NEW_LINE, ignore_index=True)
            HEAD[col[ 0]].values[i] = HEAD[col[ 0]].values[i-1]
            HEAD[col[-1]].values[i] = HEAD[col[-1]].values[i-1]
            HEAD_out[col[ 0]].values[i] = 18              # Potentialhöhe Links in m
            HEAD_out[col[-1]].values[i] = 16              # Potentialhöhe Rechts in m
            for n in range(0,ncol):
                T.append(K_Zone[n] * min([m, HEAD[col[n]].values[i-1]]))                     #Transmissivität
            for n in range(1,ncol-1):
                HEAD[col[n]].values[i]=(2*T[n]*T[n-1]/(T[n]+T[n-1])*HEAD[col[n-1]].values[i-1]+2*T[n]*T[n+1]/(T[n]+T[n+1])*HEAD[col[n+1]].values[i-1]+RCH_Zone[n]*dx**2)/(2*T[n]*T[n-1]/(T[n]+T[n-1])+2*T[n]*T[n+1]/(T[n]+T[n+1]))
                HEAD_out[col[n]].values[i] = "%5.2f"% (HEAD[col[n]].values[i])
            with output:
                clear_output()
                print(HEAD_out[1:])              
            with out_plot:
                fig = plt.figure(figsize=(4,4))   
                ax = fig.add_subplot(1, 1, 1)
                clear_output()
                ax.plot(HEAD[-1:].T)
                plt.show()
                      
            return HEAD, HEAD_out, i

def Iteration(_):
    global HEAD, HEAD_out, i
    HEAD,HEAD_out,i = iterator(HEAD=HEAD,HEAD_out=HEAD_out,i=i)

button.on_click(Iteration)
box = HBox([VBox([button, output]), out_plot])
box

<hr>
&copy; 2021 | Thomas Reimann u. a.
<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img style="float: right" alt="Creative Commons Lizenzvertrag" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a>